In [1]:
import random 
import pickle
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM,Dense,Activation, Bidirectional,Embedding
from tensorflow.keras.optimizers import RMSprop,Adam

In [2]:
data=pd.read_csv('files.csv')


In [3]:
text=list(data.title.values)
joinedtxt=" ".join(text)

In [6]:
partxt=joinedtxt[:100000]

In [7]:
tokenizer=RegexpTokenizer(r"\w+")
tokens=tokenizer.tokenize(partxt.lower())

In [8]:
uniqtok=np.unique(tokens)
unitokindex={token: idx for idx, token in enumerate(uniqtok)}

In [10]:
nwords=3
inputwords=[]
nextword=[]

for i in range(len(tokens)-nwords):
    inputwords.append(tokens [i:i + nwords])
    nextword.append(tokens[i + nwords])

In [11]:
x=np.zeros((len(inputwords),nwords,len(uniqtok)),dtype=bool)
y=np.zeros((len(nextword),len(uniqtok)),dtype=bool)

In [12]:
for i , words in enumerate(inputwords):
    for j, word in enumerate(words):
        x[i,j,unitokindex[word]]=1
    y[i,unitokindex[nextword[i]]]=1


In [13]:
model=Sequential()
model.add(LSTM(128, input_shape=(nwords,len(uniqtok)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(uniqtok)))
model.add(Activation("softmax"))

In [14]:
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(learning_rate=0.01),metrics=["accuracy"])
model.fit(x,y,batch_size=128,epochs=30,shuffle=True)

Epoch 1/30
147/147 [==============================] - 14s 64ms/step - loss: 6.4462 - accuracy: 0.0487
Epoch 2/30
147/147 [==============================] - 9s 60ms/step - loss: 6.1093 - accuracy: 0.0641
Epoch 3/30
147/147 [==============================] - 8s 56ms/step - loss: 5.8282 - accuracy: 0.0794
Epoch 4/30
147/147 [==============================] - 8s 56ms/step - loss: 5.5749 - accuracy: 0.1033
Epoch 5/30
147/147 [==============================] - 9s 60ms/step - loss: 5.3569 - accuracy: 0.1177
Epoch 6/30
147/147 [==============================] - 9s 58ms/step - loss: 5.1504 - accuracy: 0.1376
Epoch 7/30
147/147 [==============================] - 9s 58ms/step - loss: 4.9445 - accuracy: 0.1525
Epoch 8/30
147/147 [==============================] - 8s 58ms/step - loss: 4.7273 - accuracy: 0.1750
Epoch 9/30
147/147 [==============================] - 9s 58ms/step - loss: 4.5041 - accuracy: 0.1977
Epoch 10/30
147/147 [==============================] - 8s 58ms/step - loss: 4.2714 - accur

In [15]:
model.save("mymodel.h5")

C:\Users\Tak Dong Kyung\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
model=load_model("mymodel.h5")

In [23]:
def predictnextword(input_text,nbest):
    input_text=input_text.lower()
    x=np.zeros((1,nwords,len(uniqtok)))
    for i , word in enumerate(input_text.split()):
        x[0,i,unitokindex[word]]=1
    prediction= model.predict(x)[0]
    return np.argpartition(prediction, -nbest)[-nbest:]


In [28]:
str=input("Enter the initials of the sentence \n")
possible=predictnextword(str,8)
print([uniqtok[idx] for idx in possible])

Enter the initials of the sentence 
how is this
1/1 [==============================] - 0s 40ms/step
['is', 'more', 'husband', 'attention', 'name', 'doubt', 'assistant', 'to']
